## DeepLearning Model

#### Import libraries

In [21]:
import joblib
import pandas as pd
import sklearn
import sklearn.datasets
from keras.models import Sequential
from keras.layers import Dense

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)

tr_features = pd.read_csv('train_features.csv')
tr_labels = pd.read_csv('train_labels.csv', header=None)
test_features = pd.read_csv('test_features.csv')
test_labels = pd.read_csv('test_labels.csv', header=None)

val_features = pd.read_csv('val_features.csv')
val_labels = pd.read_csv('val_labels.csv', header=None)

In [22]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [23]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(tr_features)
X_train_scaled = X_scaler.transform(tr_features)
X_test_scaled = X_scaler.transform(test_features)
X_val_scaled = X_scaler.transform(val_features)

#### Train Model

In [5]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
#sparse_categorical_crossentropy was used because of multiple levels of the y-value (0, 1, 2)
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(
X_train_scaled,
tr_labels.values,
epochs=60,
shuffle=True,
verbose=2
)

Epoch 1/60
 - 1s - loss: 0.6108 - accuracy: 0.7101
Epoch 2/60
 - 0s - loss: 0.3723 - accuracy: 0.8095
Epoch 3/60
 - 0s - loss: 0.3563 - accuracy: 0.8195
Epoch 4/60
 - 0s - loss: 0.3462 - accuracy: 0.8255
Epoch 5/60
 - 0s - loss: 0.3344 - accuracy: 0.8338
Epoch 6/60
 - 0s - loss: 0.3260 - accuracy: 0.8414
Epoch 7/60
 - 0s - loss: 0.3216 - accuracy: 0.8395
Epoch 8/60
 - 0s - loss: 0.3224 - accuracy: 0.8474
Epoch 9/60
 - 0s - loss: 0.3167 - accuracy: 0.8507
Epoch 10/60
 - 0s - loss: 0.3097 - accuracy: 0.8538
Epoch 11/60
 - 0s - loss: 0.3091 - accuracy: 0.8560
Epoch 12/60
 - 0s - loss: 0.3115 - accuracy: 0.8536
Epoch 13/60
 - 0s - loss: 0.2957 - accuracy: 0.8686
Epoch 14/60
 - 0s - loss: 0.3001 - accuracy: 0.8615
Epoch 15/60
 - 0s - loss: 0.3005 - accuracy: 0.8610
Epoch 16/60
 - 0s - loss: 0.2940 - accuracy: 0.8662
Epoch 17/60
 - 0s - loss: 0.2901 - accuracy: 0.8696
Epoch 18/60
 - 0s - loss: 0.2949 - accuracy: 0.8665
Epoch 19/60
 - 0s - loss: 0.2921 - accuracy: 0.8672
Epoch 20/60
 - 0s - l

#### Model evaluation and Prediction on test-data

In [9]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, test_labels, verbose=2)
print(
    f"result_df Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

result_df Neural Network - Loss: 0.26586413888355254, Accuracy: 0.8934953808784485


In [11]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
Actual_label = (test_labels[:20]).values.ravel()

In [24]:
result_df = pd.DataFrame({
    'Prediction': encoded_predictions,
    'Actual': Actual_label
})
result_df

,Prediction,Actual
0,2,2
1,0,0
2,2,2
3,0,0
4,0,1
5,2,2
6,1,1
7,0,0
8,2,2
9,2,2


#### Write Model

In [25]:
joblib.dump(model, 'DL_model.pkl')

['DL_model.pkl']

#### Model validation using validation-data

In [26]:
 model_val = joblib.load('DL_model.pkl')

In [27]:
model_loss, model_accuracy = model_val.evaluate(
    X_val_scaled, val_labels, verbose=2)
print(
    f"result_df Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

result_df Neural Network - Loss: 0.2779291509643986, Accuracy: 0.8891273140907288
